# Porting genome scale metabolic models for metabolomics (RECON3D)

- to make formats compatible to mummichog
- to link to a common compound table 
- from compound table, we generated predicted mass peaks based on formula

As mummichog 3 is under development, treat this as part of development.

**Use cobra to parse SBML models whereas applicable**

Not all models comply with the formats in cobra. Models from USCD and Thiele labs should comply.

**Base our code on metDataModel**

Each model needs a list of Reactions, list of Pathways, and a list of Compounds.
It's important to include with Compounds with all linked identifiers to other DBs (HMDB, PubChem, etc), and with formulae (usually charged form in these models) when available.
We can alwasy update the data later. E.g. the neural formulae can be retrieved from HMDB if linked.
Save in Python pickle and in JSON.

Shuzhao Li, 2021-05-10

In [1]:
! pip install cobra

In [3]:
# https://github.com/shuzhao-li/metDataModel/ - need v 0.3.1 for serialize functions
!pip install --upgrade metDataModel

     |████████████████████████████████| 2.2MB 5.2MB/s eta 0:00:01
  Found existing installation: metDataModel 0.3.0
    Uninstalling metDataModel-0.3.0:
      Successfully uninstalled metDataModel-0.3.0


In [4]:
from metDataModel.core import Compound, Reaction, Pathway, metabolicModel

# https://cobrapy.readthedocs.io/en/latest/io.html#SBML
import cobra

In [5]:
# cloned from
# https://github.com/VirtualMetabolicHuman
# 2021-05-08
# this is the more inclusive model. The other Recon3DModel_301 is flux constrainted.
R3D = "thiele/Recon/Current_Version/Recon3D_301_Reconstruction/Recon3D_301.xml"

model = cobra.io.read_sbml_model(R3D)

model

Name,COBRAModel
Memory address,0x07ff62e99bcf8
Number of metabolites,8399
Number of reactions,13543
Number of groups,111
Objective expression,1.0*biomass_reaction - 1.0*biomass_reaction_reverse_32a6c
Compartments,"Cytoplasm, Lysosome, Mitochondrion, Endoplasmic_reticulum, Extracellular, Peroxisome, Nucleus, Golgi, unknownCompartment4"


In [6]:
[model.metabolites[33].formula,
model.metabolites[33].charge,
 model.metabolites[33].name,
 model.metabolites[33].id,
 model.metabolites[33]._id,
 model.metabolites[33].annotation
]

['C28H44O3',
 0,
 '1-Alpha,25-Dihydroxyvitamin D2',
 '1a25dhvitd2[m]',
 '1a25dhvitd2[m]',
 {'hmdb': 'HMDB06225',
  'inchi': 'InChI=1S/C28H44O3/c1-18(9-10-19(2)27(4,5)31)24-13-14-25-21(8-7-15-28(24,25)6)11-12-22-16-23(29)17-26(30)20(22)3/h9-12,18-19,23-26,29-31H,3,7-8,13-17H2,1-2,4-6H3/b10-9+,21-11+,22-12-/t18-,19+,23-,24-,25+,26+,28-/m1/s1',
  'pubchem.compound': '9547243'}]

In [7]:
model.reactions[33]

Reaction identifier,25VITD2Hm
Name,1-Alpha-Vitamin D-25-Hydroxylase (D2)
Memory address,0x07ff64602ea58
Stoichiometry,"25hvitd2[m] + h[m] + nadph[m] + o2[m] --> 1a25dhvitd2[m] + h2o[m] + nadp[m] 25-Hydroxyvitamin D2 + Proton + Nicotinamide Adenine Dinucleotide Phosphate - Reduced + Oxygen --> 1-Alpha,25-Dihydroxyvitamin D2 + Water + Nicotinamide Adenine Dinucleotide Phosphate"
GPR,1594.1
Lower bound,0.0
Upper bound,1000.0


In [8]:
def metabolite2compound(M):
    # convert cobra Metabolite to metDataModel Compound
    Cpd = Compound()
    Cpd.src_id = M.id
    Cpd.id = M.id.split("[")[0]
    Cpd.name = M.name
    Cpd.charge = M.charge
    Cpd.charged_formula = M.formula
    Cpd.db_ids = M.annotation
    return Cpd

metabolite2compound(model.metabolites[33]).id

'1a25dhvitd2'

In [9]:
# list of Compounds
myCpds = []
anno = {}
for M in model.metabolites:
    anno[M.id.split("[")[0]] = M.annotation
    myCpds.append(metabolite2compound(M))
    
print("total, ", len(myCpds), len(anno))

total,  8399 4140


In [10]:
# de-compartmentalize metabolites 

# this overwrites repeated ids
myDict = {}
for M in myCpds: myDict[M.id] = M
print(len(myDict))

list(myDict.items())[0]

## Compounds to port
R3D_Compounds = list(myDict.values())

4140


In [11]:
## This is model dependent, as some models use symbol other than "[" !!!
def mclean(x): return x.split("[")[0]

# from metDataModel.core import Compound, Reaction, Pathway, metabolicModel
# port reactions
def port_reaction(R):
    new = Reaction()
    new.id = R.id
    new.reactants = [mclean(m.id) for m in R.reactants] 
    new.products = [mclean(m.id) for m in R.products] 
    return new

test99 = port_reaction(model.reactions[199])
[test99.id,
 test99.reactants,
 test99.products,
]

['ACHEe', ['h2o', 'ach'], ['h', 'ac', 'chol']]

In [16]:
# this is the compartmentalized version of reactions
## Reactions to port
R3D_Reactions = []
for R in model.reactions:
    R3D_Reactions.append( port_reaction(R) )
    
print(len(R3D_Reactions))


13543


In [13]:
# pathways, using group as pathway from RECON3D. Other models may use subsystem etc.

def port_pathway(P):
    new = Pathway()
    new.id = P.id
    new.source = ['RECON3D',]
    new.name = P.name
    new.list_of_reactions = [x.id for x in P.members]
    return new

p = port_pathway(model.groups[33])

[p.id, p.name, p.list_of_reactions]

['group34',
 'Glycine, serine, alanine, and threonine metabolism',
 ['2AMACHYD',
  'AACTOOR',
  'ALASm',
  'AOBUTDsm',
  'BETALDHxm',
  'BHMT',
  'CHOLD2m',
  'DMGDHm',
  'GCC2am',
  'GCC2bim',
  'GCC2cm',
  'GCCam',
  'GCCbim',
  'GCCcm',
  'GHMT2rm',
  'GLYATm',
  'GLYOp',
  'GNMT',
  'PGCD',
  'SARCOXp',
  'SERHL',
  'SPTix',
  'r0160',
  'r0552',
  'r0553',
  'RE2111M',
  'RE2117M',
  'RE2427M',
  'RE2428M',
  'RE2429M',
  'GLYACm',
  'THRACm',
  'ACHOMm',
  'GHMT2r',
  'PSERT',
  'PSP_L',
  'THRS',
  'THRD_L',
  'OBDHc',
  'PPIOGLYc',
  'TIGGLYc',
  'HEXGLYc',
  'SUBGLYc',
  'HMR_4284',
  'HMR_4466',
  'HMR_4696',
  'HMR_4700',
  'HMR_7703',
  'HMR_9718']]

In [14]:
## Pathways to port
R3D_Pathways = []
for P in model.groups:
    R3D_Pathways.append(port_pathway(P))

len(R3D_Pathways)

111

In [19]:
note = """RECON3D cloned from https://github.com/VirtualMetabolicHuman, 2021-05-08.
Compounds are decompartmentalized, but Reactions are not. 
The redundant metabolites will be merged ad hoc when pathways and reactions are pulled.
"""

## metabolicModel to export
MM = metabolicModel()
MM.id = 'az_RECON3D_20210510'
MM.meta_data = {
            'species': 'human',
            'version': '',
            'sources': ['https://github.com/VirtualMetabolicHuman, retrieved 2021-05-08'],
            'status': '',
            'last_update': '20210510',
            'note': note,
        }
MM.list_of_pathways = [P.serialize() for P in R3D_Pathways]
MM.list_of_reactions = [R.serialize() for R in  R3D_Reactions]
MM.list_of_compounds = [C.serialize() for C in R3D_Compounds]

In [20]:
# check output
[
MM.list_of_pathways[:2],
MM.list_of_reactions[:2],
MM.list_of_compounds[100:102],
]

[[{'id': 'group1',
   'name': 'Alanine and aspartate metabolism',
   'list_of_reactions': ['AGTim',
    'AGTix',
    'ARGSS',
    'ASNNm',
    'ASNS1',
    'ASPNATm',
    'ASPTAm',
    'DASPO1p',
    'NACASPAH',
    'RE1473C',
    'RE2031M',
    'RE2642C',
    'ALAR',
    'ASPTA',
    'r0127',
    'ARGSL']},
  {'id': 'group2',
   'name': 'Alkaloid synthesis',
   'list_of_reactions': ['COKECBESr',
    'ECGISOr',
    'EGMESTr',
    'NMPTRCOX',
    'PECGONCOATr']}],
 [{'id': '10FTHF5GLUtl',
   'reactants': ['10fthf5glu'],
   'products': ['10fthf5glu']},
  {'id': '10FTHF5GLUtm',
   'reactants': ['10fthf5glu'],
   'products': ['10fthf5glu']}],
 [{'id': 'nrpphr',
   'name': 'Norepinephrine',
   'identifiers': {'hmdb': 'HMDB00216',
    'inchi': 'InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-5/h1-3,8,10-12H,4,9H2/p+1/t8-/m0/s1',
    'kegg.compound': 'C00547',
    'pubchem.compound': '439260'},
   'neutral_formula': '',
   'charge': 1,
   'charged_formula': 'C8H12NO3',
   'neutral_mono_mass': 0

In [21]:
import pickle

# pickled object can be imported later, and for Database upload
with open('metabolicModel_RECON3D_20210510.pickle', 'wb') as f:
    pickle.dump(MM.serialize(), f, pickle.HIGHEST_PROTOCOL)

In [22]:
import json

s = json.JSONEncoder().encode( MM.serialize() )
with open("metabolicModel_RECON3D_20210510.json", "w") as O:
    O.write(s)

## Summary

This ports reactions, pathways and compounds. Gene and enzyme information is not included. They should be when someone has time to do it.

The exported pickle can be re-imported and uploaded to Database easily.

This notebook, the pickle file and the JSON file go to GitHub repo (https://github.com/shuzhao-li/Azimuth).